這邊我們嘗試用 R 的 library 來體驗使用 Decision Tree 解分類問題

In [ ]:
#install.package("rpart")
library(rpart)

In [ ]:
#vignette(package = "rpart")

In [ ]:
# 這邊我們載入一個關於 C 期前列腺癌的研究數據，其中記錄著 146 位病患的資訊。
data(stagec)
colnames(stagec)

In [ ]:
# 建立一個決策樹模型
cfit <- rpart(pgstat ~ age + eet + g2, data = stagec, method = "class")

In [ ]:
# 請參照 rpart 的 help 說明 `pgstat ~ age + eet + g2`是對應到`rpart` 函數的哪一個參數呢？
# 再參考關於method參數的說明，其中解釋了rpart是如何依照 formula 中選擇的變數形態來智慧的選擇預設的method
# 請觀察 stagec$pgstat 的型態，請問如果我們沒有指定method 的話，rpart會用哪一種method參數來運作？

In [ ]:
# 輸入`cfit`來看看rpart 的結果
cfit

In [ ]:
# 這裡的loss代表的是錯誤的label的個數 ，俗稱0/1 loss。 
# 在第一個node，也就是root之中，cfit對`stagec$pgstat` 的預測是0。
# 請各位計算`stagec$pgstat` 中非 0 的病患總數。看看是不是和 root 中顯示的 loss相同
sum(stagec$pgstat != 0)

In [ ]:
# 接著，讓我們畫出cfit
plot(cfit)
text(cfit)

In [ ]:
# 用預設的圖有時候顯示得不完整，需要調整，載入專門的套件來看
#install.packages("rpart.plot")
library(rpart.plot)
rpart.plot(cfit)

In [ ]:
# 利用建立好的模型進行預測
predict(cfit, stagec)

In [ ]:
# 在前面的 vignette 中，Chapter 3.1 的章節說明了如何建構一個 decision tree
# 裡面解釋了何謂 prior、loss和splitting
# rpart 的參數`parms` 裡面可以設定和method相關的參數
# 請問根據`rpart` 的說明文件 (請參閱Arguments 底下的parms)
# 當method為class 時 (classificationsplitting)，預設的 prior 為何？
# 預設的splitting index 又為何？

In [ ]:
# 這其實是受到`control` 這個參數的影響，改變切點設定可以改變結果
rpart(pgstat ~ age, data = stagec, method = "class", control = rpart.control(minsplit = 1))

In [ ]:
# 這邊我們嘗試改變獲取 information 增益的方法
rpart(pgstat ~ age, data = stagec, method = "class", 
      parms = list(split = "information"), control = rpart.control(minsplit=1))

In [ ]:
# 下面我們用 mlbench 資料來進行一個練習
#install.package("mlbench", "2.1.1")
library(mlbench)

In [ ]:
# 計算 Logarithmic Loss 的函式
logloss <- function(y, p, tol = 1e-4) {
  # tol 的用途是避免對0取log所導致的數值問題
  p[p < tol] <- tol
  p[p > 1 - tol] <- 1-tol
  -sum(y * log(p) + (1 - y) * log(1-p))
}
data(Ionosphere)
test.i <- c(4L, 6L, 9L, 13L, 14L, 22L, 31L, 33L, 50L, 52L, 61L, 63L, 68L, 
            79L, 91L, 99L, 119L, 135L, 154L, 155L, 160L, 162L, 166L, 194L, 
            200L, 219L, 233L, 236L, 237L, 242L, 244L, 248L, 250L, 257L, 261L, 
            276L, 278L, 283L, 292L, 310L, 312L, 315L, 319L, 323L, 325L, 327L, 
            335L, 337L, 338L, 344L)
df.test <- Ionosphere[test.i,-2]
train.i <- setdiff(seq_len(nrow(Ionosphere)), test.i)
df.train <- Ionosphere[train.i,-2]

# 請利用rpart，從df.train上學出一個模型
# 該模型在df.test上的logloss需要小於12
model <- rpart(Class ~ ?, data = df.train, control = rpart.control(minsplit=?))

p <- predict(model, df.test)[,"good"]
logloss(df.test$Class == "good", p) < 12

In [ ]:
這邊我們嘗試手刻 kNN 算法來解決分類問題

In [ ]:
# 準備需要的資料
data(iris3)
assign("X.train", rbind(iris3[1:25,,1], iris3[1:25,,2], iris3[1:25,,3]))
assign("y.train", rep(c("setosa", "versicolor", "virginica"), each = 25))
assign("X.test", rbind(iris3[26:50,,1], iris3[26:50,,2], iris3[26:50,,3]))
assign("y.test", rep(c("setosa", "versicolor", "virginica"), each = 25))

library(dplyr)

In [ ]:
# 請觀察 X.train、y.train、X.test 以及 y.test 分別有些什麼資料
X.test

In [ ]:
# 首先將 X.train 與 X.test 合併，計算點與點之間的距離
X <- rbind(X.train, X.test)
d <- dist(X) %>%
as.matrix()

In [ ]:
# 將 test 資料集中各點，與 training 資料集中各點的距離抽出
d.train = d[1:75, 76:150]

In [ ]:
a <- c(1,7,3,2,5)
sort(a)
order(a)

In [ ]:
temp = c("a","b","c","b","b")
table(temp) %>% sort(decreasing = T) %>% names %>% '['(1)

In [ ]:
s = d.train[,1]
i5 = order(s)[1:5]
table(y.train[i5]) %>% sort %>% names %>% '['(1)

In [ ]:
# 利用 which.min 找出測試資料集中各點距離訓練資料集中最接近的點
# 這邊利用 apply 對每個點計算 which.min
i.1nn <- apply(d.train, 2, which.min)
c.5nn <- apply(d.train, 2, function(s) {
    i5 = order(s)[1:5]
    table(y.train[i5]) %>% sort %>% names %>% '['(1)
})
#head(c.5nn)
table(c.5nn, y.test)

In [ ]:
i.1nn

In [ ]:
# 接著，利用 y.train[i.1nn]，我們就可以取得對應的training data point的類別
# 也就是說 y.train[i.1nn]就是1NN的分類結果。
table(y.train[i.1nn], y.test)

In [ ]:
# 請比較分類結果與標籤，並嘗試計算訓練資料集中分類的準確度

In [ ]:
# 上述方法要計算 knn 是比較困難的
# 這邊我們直接採用 R 語言中有實作 knn 的套件
install.packages("class")
library(class)

In [ ]:
# 請參閱 knn 的 help，並嘗試下列程式碼
result.1nn = knn(X.train, X.test, y.train, k = 1)

In [ ]:
# 請比較套件計算的結果與手算的結果
table(result.1nn, y.test)

In [ ]:
# install.packages("mlbench", "2.1.1")
library(mlbench)

data(Ionosphere)
test.i <- c(4L, 6L, 9L, 13L, 14L, 22L, 31L, 33L, 50L, 52L, 61L, 63L, 68L, 
  79L, 91L, 99L, 119L, 135L, 154L, 155L, 160L, 162L, 166L, 194L, 
  200L, 219L, 233L, 236L, 237L, 242L, 244L, 248L, 250L, 257L, 261L, 
  276L, 278L, 283L, 292L, 310L, 312L, 315L, 319L, 323L, 325L, 327L, 
  335L, 337L, 338L, 344L)
df.test <- Ionosphere[test.i,-2] # remove V2
X.test <- df.test[,-34]
y.test <- df.test$Class
train.i <- setdiff(seq_len(nrow(Ionosphere)), test.i)
df.train <- Ionosphere[train.i,-2]
X.train <- df.train[,-34]
y.train <- df.train$Class

# 以下程式碼示範用euclidean distance來計算1NN的分類結果。
df <- rbind(X.train, X.test)
d <- dist(df, method = "euclidean")
m <- as.matrix(d)
i <- seq_len(nrow(df.train))
j <- nrow(df.train) + seq_len(nrow(df.test))
m2 <- m[i,j]
i.1nn <- apply(m2, 2, which.min)
accuracy <- mean(y.test == y.train[i.1nn])

# 請同學修改上述程式碼中，dist函數的參數
# 讓R使用其他的「距離定義」，找出讓 accuracy 的準確度超過0.95的結果


In [ ]:
accuracy
table(y.test, y.train[i.1nn])